# Homework 3: Logistics Regression
Implement everywhere that has an ellipsis (`...`).

In [118]:
import numpy as np

### Load dataset

In [119]:
from sklearn.datasets import load_iris
data = load_iris()
X, Y = data.data, data.target

In [120]:
# only classify class 1 vs. class 2
indices = np.where(Y == 0)
Y = np.delete(Y, indices)
X = np.delete(X, indices, axis=0)

X = np.c_[np.ones((len(X),1)),X]

### Data normalization
Remember: normalization is done per feature, not all at once. Also, remember that Y is [1, 2] instead of [0, 1].

In [121]:
# Write your code here. Don't forget the bias!
X = (X - np.mean(X))/np.std(X)
Y = (Y - np.mean(Y))/np.std(Y)

### Define classification model
Create parameters here. Initialize with zeros. In case you forgot: $Y = \sigma(X\Theta)$.

In [122]:
weight = np.zeros((5))

### Define evaluation metrics

In [123]:
def accuracy(X, Y, weight):
    '''
    Evaluate the model, represented by `weight`, with data (X, Y).
    
    Input:
        X:      data features
        Y:      data labels
        weight: model weights
    Output:
        Model accuracy on input data.
    '''
    
    # implement your code here
    correctGuessNum = 0
    for index, x in enumerate(X):
        prediction = np.dot(x, weight)
        sigmoid = 1/(1 + np.exp(-prediction))
        prediction = -1 if sigmoid < 0.5 else 1
        actual = Y[index]
        if (actual == prediction):
            correctGuessNum += 1
    return correctGuessNum / len(Y)

### Hyperparameters

In [124]:
# learning rate
alpha = 1e-2
# epochsNotebooks
epoch = 10000

### Train the model.

In [125]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [126]:
m = len(Y)

for i in range(epoch):
    prediction = np.array([ sigmoid(x) for x in np.dot(X, weight)])
    delta = (1/m) * alpha * ( X.T.dot( prediction - Y ))
    weight = weight - delta

In [127]:
# just to double check
assert np.allclose(weight, np.array([0.12047504, -0.44156746, -0.89309501, 2.965364, 3.3427994]))

AssertionError: 

In [128]:
# model accuracy
accuracy(X, Y, weight)

0.58

### (Optional) How low can you go?
Do anything you want to get the best performance out of the training set. For once, let's overfit to your heart's content.

In [133]:
# do some fun code here and try to match this :)
...

In [84]:
print('Weights:', weight)
y = 1 / (1 + np.exp(-X @ weight))
loss = -np.sum(Y * np.log(y) + (1 - Y) * np.log(1 - y))
print('Loss:', loss)
print('Accuracy:', accuracy(X, Y, weight))

Weights: [-0.77510674 -0.75821084 -0.94407505  1.90818808  1.82476014]
Loss: 2.803551934596323
Accuracy: 0.99
